## Multilayer Perceptron (MLP) based on TensorFlow

Multi-class classification problem - using a MLP with configurable number of hidden neurons - with a configurable number of classes (up to 10). It selects them from the (Fashion-)MNIST dataset, splits it up into a train and test part, does normalisation and then trains a classifier using softmax.

Both datasets consist of images with 28x28 = 784 pixel each. The features refer to these pixel values of the images.

You can choose MNIST or Fashion-MNIST data in cell [2]

We use the Keras TensorFlow API in particular the Sequential model, which provides all required layer types [tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras).

In [ ]:
import torch
import torchvision
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard 
import numpy as np
import matplotlib.pyplot as plt
import time

from utils import read_data, plot_img, plot_tiles

In [ ]:
#note the path in read_data for the data, which points to ../week1/data
x, y, labels_map = read_data('fashionMNIST', storage_path='../week1/data') #MNIST or fashionMNIST
x = x.numpy()
y = y.numpy()

In [ ]:
plot_img(x[0])

In [ ]:
#append rows x cols tiles of images
rows = 8
cols = 18
#figure size can be set
fig_size = [8,8]

plot_tiles(x, rows, cols, fig_size)

In [ ]:
#choose a given class 0..9
digit  = 0

plot_tiles(x[y==digit], rows, cols, fig_size)
print(labels_map[digit])

In [ ]:
#select the classes for your training and test set, select train and test split and to normalization
def prepare_data(classes, train_size=0.8, min_max_normalise=1, flatten=1):
    """
    prepare the data for training

    Arguments:
    classes -- list of classes to use for training (at least two classes must be given)
    train_size -- fraction of train image size
    min_max_normalise -- whether to do min-max-normalisation (1) or rescaling (0)
    flatten -- whether to flatten the 28x28 image to single row (=1); otherwise a new dimension is added at axis=1 (to be compatible with cnn)
    """

    if len(classes) < len(labels_map):
        for label in classes:
            print('labels chosen are: %r' % labels_map[label])

    ind_sel = np.isin(y, classes)
    x_sel = x[ind_sel,:].copy()
    y_sel = y[ind_sel].copy()

    #replace the labels such that they are in successive order
    for i0 in range(0,len(classes)):
        if i0 != classes[i0]:
            y_sel[y_sel == classes[i0]] = i0

    #we give y back as simple vector -> simplifies handling below
    #y_sel = np.reshape(y_sel, (-1,1))
    
    #do train and test split
    num_samples = x_sel.shape[0]
    max_train_ind = int(train_size*num_samples)
    indices = np.arange(num_samples)
    np.random.shuffle(indices)
    
    x_train = x_sel[indices[:max_train_ind]]
    x_test = x_sel[indices[max_train_ind:]]
    
    y_train = y_sel[indices[:max_train_ind]]
    y_test = y_sel[indices[max_train_ind:]]

    #perform normalisation, take care of converting data type to float!
    xmax, xmin = np.max(x_train), np.min(x_train)
    
    if min_max_normalise:
        x_train = 2*(x_train.astype(float) - xmin) / (xmax - xmin) - 1
        x_test = 2*(x_test.astype(float) - xmin) / (xmax - xmin) - 1
    else:
        x_train = x_train.astype(float) / xmax 
        x_test = x_test.astype(float) / xmax 

    if flatten:
        m = x_train.shape[0]
        x_train = x_train.reshape([m,-1])
        m = x_test.shape[0]
        x_test = x_test.reshape([m,-1])
    else:
        x_train = np.expand_dims(x_train,1)
        x_test = np.expand_dims(x_test,1)

    #tensorflow requires onehot data
    y_train = one_hot(y_train, len(classes))
    
    return x_train, x_test, y_train, y_test


def one_hot(y, size_out):
        """
        construct onehot vector from set of labels
        """
        m = y.shape[0]
        one_hot = np.zeros((m, size_out), dtype=float)
        one_hot[np.arange(m), y] = 1

        return one_hot

### Class NeuralNetwork

This class constructs a Multilayer Perceptron with a configurable number of hidden layers. Cost function is CE. The method $propagate()$ returns the prediction $$ \hat{y}^{(i)}=h_\theta(\mathbf{x}^{(i)}) $$ on the input data (can be a n x 784 matrix of n images) and $back\_propagate()$ determines the gradients of the cost function with respect to the parameters (weights and bias for all layers) $$ \nabla_{\mathbf{\theta}} J(\mathbf{\theta}) $$
The method $gradient\_descend()$ finally does the correction of the parameters with a step in the negative gradient direction, weighted with the learning rate $$\alpha$$ for all layers.

In [ ]:
class NeuralNetwork:
    """
    MLP class handling the layers and doing all propagation and back propagation steps
    all hidden layers are dense (with ReLU activation) and the last layer is softmax
    """
    def __init__(self, list_num_neurons, alpha):
        """
        constructor

        Arguments:
        list_num_neurons -- list of layer sizes including in- and output layer
        alpha -- learning rate (required because optimiser must be known to compile model)
        """
        self.model = tf.keras.Sequential()
        
        #we require a flatten tensor
        self.model.add(tf.keras.layers.Flatten(input_shape=(1, 28, 28)))
        
        for i0 in range(len(list_num_neurons)-1):
            self.model.add(tf.keras.layers.Dense(list_num_neurons[i0], activation='sigmoid'))
            
        #finally add softmax layer
        self.model.add(tf.keras.layers.Dense(list_num_neurons[-1], activation='softmax'))

        print(self.model.summary())
                         
        #choose the optimiser
        optimizer = tf.keras.optimizers.SGD(learning_rate=alpha)
        
        #wrap the model to a tf.function (unless run_eagerly=True)
        self.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
         
        
    def optimise(self, data, epochs, valid_size=0.2, batch_size=16, debug=0, call_backs=None):
        """
        performs epochs number of gradient descend steps and appends result to output array

        Arguments:
        data -- dictionary with NORMALISED data
        epochs -- number of epochs
        valid_size -- fraction of data used for validation
        batch_size -- size of batches (1 = SGD, 1 < .. < n = mini-batch)
        debug -- output: 0 = silent, 1 = progress bar, 2 = one line per epoch
        call_backs -- use to send output to tensorboard
        """

        #now start training (you can choose the train/validation set split)
        self.history = self.model.fit(data['x_train'], data['y_train'], 
                            validation_split=valid_size,   
                            batch_size=batch_size, epochs=epochs,
                            callbacks=call_backs, verbose = debug)
         

### Plot Function for Tensorflow

The two functions below illustrate how to access the result data without using tensorboard

In [ ]:
def plot_error(nn_instance):
    """
    analyse error as function of epochs

    Arguments:
    nn_instance -- NeuralNetwork class to plot
    """
    epochs = np.arange(len(NNet.history.history['accuracy']))
    train_acc = nn_instance.history.history['accuracy']
    val_acc = nn_instance.history.history['val_accuracy']

    plt.plot(epochs, train_acc, label="train")
    plt.plot(epochs, val_acc, label="validation")
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    xmax = epochs[-1]
    ymin = min(min(train_acc),min(val_acc))
    ymax = 1
    plt.axis([0,xmax,ymin,ymax])
    plt.legend()
    plt.show()

def plot_cost(nn_instance):
    """
    analyse cost as function of epochs

    Arguments:
    nn_instance -- NeuralNetwork class to plot
    """
    epochs = np.arange(len(nn_instance.history.history['loss']))
    train_costs = nn_instance.history.history['loss']
    val_costs = nn_instance.history.history['val_loss']

    plt.semilogy(epochs, train_costs, label="train")
    plt.semilogy(epochs, val_costs, label="validation")
    plt.ylabel('Cost')
    plt.xlabel('Epochs')
    xmax = epochs[-1]
    ymin = 1e-2
    ymax = 2
    plt.axis([0,xmax,ymin,ymax])

    plt.legend()
    plt.show()

### Sample execution of Neural Network

The cell below shows how to use the class NeuralNetwork and how to perform the optimisation. The training and test data is given as dictionary in the call to the method $optimise()$. The classes (from 2 to 10) can be chosen via the `classes` list. 

*This version is used without TensorBoard*

In [ ]:
#choose the categories
classes = [0,1,2,3,4,5,6,7,8,9]

#y_train is of type onehot (y_test not!)
x_train, x_test, y_train, y_test = prepare_data(classes, train_size=0.8, min_max_normalise=1, flatten=0)

#data is arranged as dictionary with quick access through respective keys
#tensorflow does the split in train and validate
data = {'x_train' : x_train, 'y_train' : y_train}

#choose the hyperparameters you want to use for the initialisation
size_out = len(classes)
list_num_neurons = [100, size_out]; 
learning_rate = 0.05
NNet = NeuralNetwork(list_num_neurons, learning_rate)

#choose the hyperparameters you want to use for training
epochs = 100
batchsize = 16
NNet.optimise(data, epochs, valid_size=0.2, batch_size=batchsize, debug=2)

plot_error(NNet)
plot_cost(NNet)

In [ ]:
#analyse false classified training or test images
y_pred = np.argmax(NNet.model.predict(x_test), axis=1)
false_classifications = x_test[(y_pred != y_test)]

print('test error rate: %.2f %% out of %d' % (100*false_classifications.shape[0]/y_pred.shape[0], y_pred.shape[0]))
print(false_classifications.shape)

#append rows x cols tiles of digits
rows = 7
cols = 8
#figure size can be set
fig_size = [8,8]

plot_tiles(false_classifications.reshape([-1,28,28]), rows, cols, fig_size)

#print the correct labels (for FashionMNIST)
if rows*cols < false_classifications.shape[0]:
    false_classifications_y = y_test[y_pred != y_test][:rows*cols]
else:
    false_classifications_y = np.append(y_test[y_pred != y_test], np.ones(rows*cols - false_classifications.shape[0])*-1)
print(false_classifications_y.reshape([cols,rows]).T)

In [ ]:
#visualise weights of the first layer

print('we have %r weight vectors in layer [0]' % NNet.model.layers[1].get_weights()[0].shape[1])
print('choose a suitable combination of rows and cols below to plot them')

rows = 5
cols = 20
#figure size can be set
fig_size = [14,6]

plot_tiles(NNet.model.layers[1].get_weights()[0].T.reshape([-1,28,28]), rows, cols, fig_size)

### Sample execution of Neural Network

The cell below shows how to use the class NeuralNetwork and how to perform the optimisation. The training and test data is given as dictionary in the call to the method $optimise()$. The classes (from 2 to 10) can be chosen via the `classes` list. 

*This version is used with TensorBoard*

In [ ]:
#clear everything
tf.keras.backend.clear_session()

#you may have to delete the model to start from scratch
#del model

#output is written to the folder with name defined below
name = 'fashion_mnist_experiment_tf'

tensorboard = TensorBoard(
        log_dir='./tensorboard/' + name + '/', 
        update_freq='epoch', write_graph=True,
        histogram_freq=1)


#choose the categories
classes = [0,1,2,3,4,5,6,7,8,9]

#y_train is of type onehot (y_test not!)
x_train, x_test, y_train, y_test = prepare_data(classes, train_size=0.8, min_max_normalise=1, flatten=0)

#data is arranged as dictionary with quick access through respective keys
#tensorflow does the split in train and validate
data = {'x_train' : x_train, 'y_train' : y_train}

#choose the hyperparameters you want to use for the initialisation
size_out = len(classes)
list_num_neurons = [100, size_out]; 
learning_rate = 0.05
NNet = NeuralNetwork(list_num_neurons, learning_rate)

#choose the hyperparameters you want to use for training
epochs = 100
batchsize = 16
NNet.optimise(data, epochs, valid_size=0.2, batch_size=batchsize, debug=0, call_backs=[tensorboard])